In [1]:
import beta_rec
beta_rec.__version__

'0.2.4'

In [2]:
import pandas as pd
import numpy as np

In [3]:
from beta_rec.utils.evaluation import *

In [4]:
df_pred = pd.read_csv('./pred.csv')
df_test = pd.read_csv('./test.csv')

df_test['relevance'] = df_test.relevance.astype(int)

In [5]:
df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], 
    on=['user_id', 'item_id'], 
    how='left'
)
df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]
df_pred_new.fillna(0, inplace=True)
df_pred_new.loc[df_pred_new.relevance != 0, 'relevance'] = 1
df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [6]:
df_pred_new.head()

,user_id,item_id,score,relevance
40,3,590,0.280305,0.0
41,3,1136,0.263707,0.0
42,3,920,0.231234,0.0
43,3,1246,0.188023,0.0
44,3,318,0.187111,0.0


In [7]:
df_test.head()

,user_id,item_id,rating,timestamp,relevance
0,1256,5378,5.0,2007-10-31 12:18:24,5
1,1256,778,4.5,2007-10-31 12:19:51,4
2,1256,8376,5.0,2007-10-31 12:30:59,5
3,1256,2594,5.0,2007-10-31 12:20:49,5
4,1256,30810,4.5,2007-10-31 12:32:24,4


In [8]:
precision_at_k(
    df_test,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
    k=20,
)

0.0575192096597146

In [9]:
recall_at_k(
    df_test,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
    k=20,
)

0.09632131280310642

In [15]:
map_at_k(
    df_pred_new,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
    k=20,
)

0.3193571897866005

In [19]:
ndcg_at_k(
    df_test,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
    k=20,
)

0.09326437896736813

In [12]:
auc(
    df_pred_new,
    df_pred_new,
    col_user='user_id',
    col_item='item_id',
    col_rating='relevance',
    col_prediction='score',
)

0.5305776379264664